# **Modelling and Evaluation V5**

## Objectives

* Answer business requirement 1:
    * The client aims to visually differentiate lesions. The model should be capable of reaching an accuracy of at least 70%.
<br><br>

* Answer business requirement 2:
    - The model should provide a confidence level for each prediction.
<br><br>

* Answer business requirement 3:
    - If a skin lesion is predicted as malignant with high confidence, the system should recommend immediate medical consultation.
<br><br>

* Answer business requirement 5:
    - The AI model's insights should assist healthcare professionals in making informed decisions about the treatment process.
<br><br>

* Answer business requirement 6:
    - The model's performance will be evaluated using balanced performance metrics such as F1 Score.
<br><br>
## Inputs

* inputs/skin_cancer_dataset/sorted_images/train
* inputs/skin_cancer_dataset/sorted_images/test
* inputs/skin_cancer_dataset/sorted_images/validation
* image shape embeddings

## Outputs

* Images distribution plot in train, validation, and test set.
* Image augmentation.
* Class indices to change prediction inference in labels.
* Machine learning model creation and training.
Save model.
* Learning curve plot for model performance.
* Model evaluation on pickle file.
* Prediction on the random image file.

## Additional Comments

* V5 - Xception architecture as its base with smaller image size
<br><br>

* This model can be used for image classification tasks where you have multiple classes. It leverages the power of a pre-trained Xception model and fine-tunes it with additional custom layers to suit the specific classification problem.

---

---

# Import regular packages

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import joblib

---

# Change working directory

In [ ]:
current_dir = os.getcwd()
current_dir

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

In [ ]:
current_dir = os.getcwd()
current_dir

---

## Set input directories

Set train, validation and test paths.

In [ ]:
my_data_dir = 'inputs/skin_cancer_dataset/sorted_images'
train_path = my_data_dir + '/train'
val_path = my_data_dir + '/validation'
test_path = my_data_dir + '/test'

## Set output directory

In [ ]:
version = 'modelling_evaluation_v5'
file_path = f'outputs/{version}'

if 'outputs' in os.listdir(current_dir) and version in os.listdir(current_dir + '/outputs'):
    print('Old version is already available create a new version.')
    pass
else:
    os.makedirs(name=file_path)

## Set label names

In [ ]:
train_labels = os.listdir(train_path)
print('Label for train set the images are', train_labels, 'there are', len(train_labels) )
test_labels = os.listdir(test_path)
print('Label for test set the images are', test_labels, 'there are', len(test_labels) )
val_labels = os.listdir(val_path)
print('Label for val set the images are', val_labels, 'there are', len(val_labels) )

## Set image shape

In [ ]:
image_shape = (75, 75, 3)
image_shape

In [ ]:
joblib.dump(value=image_shape ,
            filename=f"{file_path}/image_shape.pkl")

In [ ]:
import joblib
image_shape = joblib.load(filename=f"{file_path}/image_shape.pkl")
image_shape

---

## Build training, validation and test set

In [ ]:
import tensorflow as tf

img_height = 75
img_width = 75
batch_size = 32

my_seed = 123
np.random.seed(my_seed)
tf.random.set_seed(my_seed)

df_train = tf.keras.utils.image_dataset_from_directory(
    train_path,
    seed=my_seed,
    color_mode="rgb",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode="int"
)

df_val = tf.keras.utils.image_dataset_from_directory(
    val_path,
    seed=my_seed,
    color_mode="rgb",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode="int"
)

df_test = tf.keras.utils.image_dataset_from_directory(
    test_path,
    seed=my_seed,
    color_mode="rgb",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode="int"
)

class_names = df_train.class_names

normalization_layer = tf.keras.layers.Rescaling(scale=1./255, offset=0)

df_train = df_train.map(lambda x, y: (normalization_layer(x), y))
df_val = df_val.map(lambda x, y: (normalization_layer(x), y))
df_test = df_test.map(lambda x, y: (normalization_layer(x), y))


In [ ]:
y = np.concatenate([y for x, y in df_train], axis=0)
print(len(y))
print(len(df_train))
ranges = [0] * 7
for class_id in np.unique(y, axis=0):
    for cl in y:
        if (class_id == cl):
            ranges[class_id] += 1
    
print(ranges)
print(sum(ranges))

---

# Model creation

---

## ML model

### Import model packages

In [ ]:
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import Xception
import tensorflow as tf

### Model

In [ ]:
def create_tf_model(num_classes, input_shape=image_shape):
    base_model = Xception(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = True
    
    model = Sequential()
    
    model.add(base_model)

    model.add(Flatten())

    model.add(Dense(128))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Dense(num_classes))

    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

### Model Summary

In [ ]:
create_tf_model(num_classes=7).summary()

In [ ]:
model = create_tf_model(num_classes=7)

model.fit(df_train,
          epochs=30,
          validation_data=df_val,
          )

## Save model

In [ ]:
model.save(f'{file_path}/lesion_classifier_model.h5')

---

## Model Performance

---

## Model learning curve

In [ ]:
losses = pd.DataFrame(model.history.history)

sns.set_style("whitegrid")
losses[['loss', 'val_loss']].plot(style='.-')
plt.title("Loss")
plt.savefig(f'{file_path}/model_training_losses.png',
            bbox_inches='tight', dpi=150)
plt.show()

print("\n")
losses[['accuracy', 'val_accuracy']].plot(style='.-')
plt.title("Accuracy")
plt.savefig(f'{file_path}/model_training_acc.png',
            bbox_inches='tight', dpi=150)
plt.show()

## Model Evaluation

### Load Model

In [ ]:
from keras.models import load_model
model = load_model(f"{file_path}/lesion_classifier_model.h5")

### Evaluate Model

In [ ]:
evaluation = model.evaluate(df_test)

### Save evaluation pickle

In [ ]:
joblib.dump(value=evaluation,
            filename=f"{file_path}/evaluation.pkl")

## Predict on new data

In [ ]:
import random
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import softmax

target_map = {v: k for v, k in enumerate(class_names)}

f, ax = plt.subplots(7, 5)  
f.set_size_inches(20, 20) 

correct_counts = {label: 0 for label in class_names}
total_counts = {label: 0 for label in class_names}

for row, label in enumerate(class_names):
    image_files = os.listdir(test_path + '/' + label)

    random_indices = random.sample(range(len(image_files)), 5)

    for col, idx in enumerate(random_indices):
        pil_image = image.load_img(test_path + '/' + label + '/' + image_files[idx], target_size=image_shape, color_mode='rgb')
        
        my_image = image.img_to_array(pil_image)
        my_image = np.expand_dims(my_image, axis=0)/255
        
        pred_logits = model.predict(my_image)[0]
        pred_proba = softmax(pred_logits)
        
        pred_class_index = np.argmax(pred_proba)
        pred_class = target_map[pred_class_index]
        
        probabilities_text = '\n'.join([f"{target_map[i]}: {p:.2f}" for i, p in enumerate(pred_proba)])

        is_correct = pred_class == label
        if is_correct:
            correct_counts[label] += 1
        total_counts[label] += 1
        
        is_correct = 'Correct' if pred_class == label else 'Incorrect'
        
        ax[row, col].imshow(pil_image)
        ax[row, col].set_title(f"pred: {pred_class}\nactual: {label}\n{is_correct}\n{probabilities_text}")
        

plt.tight_layout()
plt.show()

## Predict on new data summary

In [ ]:
total_correct = 0
for label in train_labels:
    total_correct = total_correct + correct_counts[label] 
    accuracy = (correct_counts[label] / total_counts[label]) * 100 if total_counts[label] > 0 else 0
    print(f"Accuracy for class {label}: {accuracy:.2f}% ({correct_counts[label]}/{total_counts[label]})")
    
total_accuracy = (total_correct / 35) * 100 if total_correct > 0 else 0
print()
print(f"Total correct: {total_correct}")
print(f"Accuracy total: {total_accuracy:.2f}% ({total_correct}/35)")

## Predict on new data single

In [ ]:
import random
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import softmax

from tensorflow.keras.preprocessing import image

pointer = 2
label = train_labels[1]  # select

pil_image = image.load_img(test_path + '/' + label + '/'  + os.listdir(test_path+'/' + label)[pointer], target_size=image_shape, color_mode='rgb')
print(f'Image shape: {pil_image.size}, Image mode: {pil_image.mode}')
print(f'{label}')
pil_image

In [ ]:
my_image = image.img_to_array(pil_image)
my_image = np.expand_dims(my_image, axis=0)/255
print(my_image.shape)

In [ ]:
my_image = image.img_to_array(pil_image)
my_image = np.expand_dims(my_image, axis=0)/255

pred_logits = model.predict(my_image)[0]
pred_proba = softmax(pred_logits)

pred_class_index = np.argmax(pred_proba)
pred_proba_max = np.max(pred_proba) 

target_map = {v: k for v, k in enumerate(class_names)}

pred_class = target_map[pred_class_index]

print(f"Predicted Class: {pred_class}")
print(f"Probability: {pred_proba_max:.2f}")

for i, prob in enumerate(pred_proba):
    print(f"Probability of {target_map[i]}: {prob:.2f}")

## Classification report 

In [ ]:
import numpy as np

all_features = []
all_labels = []

for features, labels in df_test:
    all_features.append(features.numpy())
    all_labels.append(labels.numpy())

all_features = np.concatenate(all_features, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

print("Features shape:", all_features.shape)
print("Labels shape:", all_labels.shape)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import os

X_train = all_features
y_true = all_labels

y_pred = model.predict(X_train)

y_pred_labels = np.argmax(y_pred, axis=1)

unique_labels = np.unique(np.concatenate((y_true, y_pred_labels)))

unique_class_names = [class_names[label] for label in unique_labels]

cm = confusion_matrix(y_true, y_pred_labels, labels=unique_labels)

plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=unique_class_names, yticklabels=unique_class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig(f'{file_path}/confusion_matrix.png')
plt.show()

class_report = classification_report(y_true, y_pred_labels, target_names=unique_class_names)

with open(os.path.join(f'{file_path}/classification_report.txt'), 'w') as file:
    file.write(class_report)

print(class_report)

## AUC-ROC and precision-recall curve

In [ ]:
import numpy as np
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from tensorflow.keras.models import load_model
import os

model = load_model(f'{file_path}/lesion_classifier_model.h5')

predictions = model.predict(df_val.map(lambda x, y: x))

y_true = np.concatenate([y for _, y in df_val], axis=0)

y_true_binarized = label_binarize(y_true, classes=range(len(class_names)))

fpr = dict()
tpr = dict()
roc_auc = dict()

n_classes = len(class_names)

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_binarized[:, i], predictions[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure()
for i, class_name in enumerate(class_names):
    plt.plot(fpr[i], tpr[i], label=f'ROC curve of {class_name} (area = {roc_auc[i]:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.savefig(f'{file_path}/roc_curves.png')
plt.show()

precision = dict()
recall = dict()
average_precision = dict()

for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_true_binarized[:, i], predictions[:, i])
    average_precision[i] = average_precision_score(y_true_binarized[:, i], predictions[:, i])

plt.figure()
for i, class_name in enumerate(class_names):
    plt.plot(recall[i], precision[i], label=f'Precision-Recall curve of {class_name} (area = {average_precision[i]:.2f})')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower right")
plt.savefig(f'{file_path}/precision_recall_curves.png')
plt.show()

---

---

NOTE

The overall accuracy of the model on the test set is 81%. This means that the model correctly predicted the skin condition 81% of the time.

When looking at individual classes, the model performed the best in identifying Melanocytic Nevi (nv) with a precision of 0.90 and recall of 0.92. Precision is the proportion of true positive identifications (i.e., the model correctly identified the condition) among all positive identifications, and recall is the proportion of true positive identifications among all actual positives.

The model also performed reasonably well in identifying Vascular Lesions (vasc) with a precision of 0.73 and a recall of 0.83. For Dermatofibroma (df), the precision is 0.57 and recall is 0.71.

However, the model had a relatively lower performance in identifying Actinic Keratosis (akiec) and Melanoma (mel). For akiec, the precision is 0.67 and the recall is 0.36, while for mel, the precision is 0.65 and the recall is 0.43.

In general, the model showed good potential in diagnosing skin conditions but further improvements, especially for Actinic Keratosis and Melanoma classes, could enhance its diagnostic capability.

---